In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import numpy as np
import datetime
import cv2
import time
from numpy.linalg import norm 

import matplotlib.pyplot as plt
import scipy, scipy.io.wavfile
from math import pi, cos, sin
import os

import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/home/cat/Downloads/spinnaker_python-1.27.0.48-Ubuntu18.04-cp37-cp37m-linux_x86_64/Examples/Python3/')


Autosaving every 180 seconds


In [7]:
# Ari's function
def get_cmp(stream_segment, templates, max_spikes, thresh):
    
    stream = stream_segment.copy()

    chosen_L2 = [] 
    
    T = stream.shape[0]-1#-55
    
    cmp = {}
    c0=1
    while True:
        print (c0)
        #convs = np.zeros([8,T+1])
        convs = np.zeros([8,T+1])
        for i in range(8):
            convs[i,:] = 2*np.correlate(stream, templates[i,:], 'same') - norm(templates[i,:])**2
        
        amax= convs.argmax(1)
        tte = convs[range(8),amax].argmax()
        loc = amax[tte]

        # exit condition based on energy threshold
        maxval = convs[:,amax[tte]].max(0)
        if maxval<thresh:
            print ("Threshold reachec, exiting ", maxval)
            break
        
        #if c0 == max_spikes: 
        #    break   
    
        chosen_L2.append(convs[tte,loc])    
        
        start = -50
        end = start+101

        #fig=plt.figure()
        #plt.plot(stream[loc+start:loc+end])
        stream[loc+start:loc+end] -= templates[tte,:]
        #plt.plot(stream[loc+start:loc+end])
        c0+=1
        
        cmp[loc] = tte
        plt.show()
        
        #if c0>3:
        #    return
    
    lc = len(cmp)
    cmp_ar = np.zeros([lc,2],dtype=np.int32)
    cmp_ar[:,0] = np.sort(list(cmp.keys()))

    for i in range(lc):
        cmp_ar[i,1] = cmp[cmp_ar[i,0]]
                
    # shift times
    cmp_ar[:,0]-=50
    return cmp_ar, stream


In [8]:
# load data
fname = '/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz.npy'
data= np.load(fname)
templates = np.load('/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz_templates.npy')
spikes = np.load('/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz_times.npy',allow_pickle=True)

# convert spike trains to usual 2-col format
spike_train = np.zeros((0,2),'int32')
for k in range(len(spikes)):
    ids = np.zeros(spikes[k].shape[0], 'int32')+k
    temp = np.vstack((spikes[k],ids)).T
    spike_train = np.vstack((spike_train,temp))

print (spike_train)

# set some thresholds
max_spikes = 1E8
thresh = 10 #energy based threshold

# flatten ends of data to avoid subtracting from ends and causing error
buffer = 200

# make a test datastream
start = 0 + buffer
end = 200000 - buffer
data[:buffer]=0
stream_segment = data[:end]
stream_segment[-buffer:]=0

# add correlated noise
corr_noise = np.load('/media/cat/4TBSSD/data/synthetic/run11/correlated_noise.npy')
print (corr_noise.shape)

stream_segment+=corr_noise[100:100+stream_segment.shape[0],10]

# visualize stream if required
#plt.plot(stream_segment)

# grab spikes from streamed_segment
idx = np.where(np.logical_and(spike_train[:,0]>start, spike_train[:,0]<=end))[0]
spk_gt = spike_train[idx]
print ("ground truth spike train for segment: ", spk_gt.shape)


[[    943       0]
 [   1687       0]
 [   1947       0]
 ...
 [8996077       7]
 [8996652       7]
 [8997954       7]]
(300000, 384)
ground truth spike train for segment:  (2075, 2)


In [11]:
fname_out = fname[:-4]+'_res.npz'

if os.path.exists(fname_out)==False:
    # run CMP
    spk_cmp, stream_deconv = get_cmp(stream_segment, templates, 
                                    max_spikes,
                                   thresh)
    print (spk_cmp)

    np.savez(fname_out,
            spk_cmp=spk_cmp,
             stream_deconv=stream_deconv)

else:
    data = np.load(fname_out)
    spk_cmp = data['spk_cmp']
    stream_deconv = data['stream_deconv']
    
fig=plt.figure()
plt.plot(stream_segment)
plt.plot(stream_deconv)
plt.plot(corr_noise[100:100+stream_segment.shape[0],10],color='green')
plt.show()

In [5]:
# fname_out = fname[:-4]+'_res.npz'

# np.savez(fname_out,
#             spk_cmp=spk_cmp,
#              stream_deconv=stream_deconv)

In [12]:
# compute errors per unit and plot against time
ptps = templates.ptp(1)

fnames = ['/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz_res_no_noise.npz',
         '/media/cat/4TBSSD/data/synthetic/ari/data_raw_30000hz_300sec_20hz_to_50hz_res.npz']

clrs = ['blue','red']
fig = plt.figure()
ctr=0
labels = ["No Correlated Noise Data","Correlated Noise Data"]
for fname in fnames:
    
    data = np.load(fname)
    spk_cmp = data['spk_cmp']
    stream_deconv = data['stream_deconv']
    
    # count # of spikes in each cluster
    n_cmp_spikes = []
    for k in range(templates.shape[0]):
        n_cmp_spikes.append(np.where(spk_cmp[:,1]==k)[0].shape[0])
    n_cmp_spikes = np.array(n_cmp_spikes)

    # match units
    matches_tp = np.zeros(templates.shape[0])
    n_spikes_gt = np.zeros(templates.shape[0],'float32')
    for k in range(spk_gt.shape[0]):
        time = spk_gt[k,0]
        id_ = spk_gt[k,1]

        n_spikes_gt[id_]+=1

        # pull out deconvolved spike train for particular unit
        idx = np.where(spk_cmp[:,1]==id_)[0]

        if idx.shape[0]==0:
            continue
        # match to within 5 timesteps
        if np.min(np.abs(spk_cmp[idx] - time))<5:
            matches_tp[id_]+=1

    # reorder by ptp
    idx = np.argsort(ptps)
    matches_tp = matches_tp[idx]
    n_cmp_spikes = n_cmp_spikes[idx]

    ax1=plt.subplot(121)
    # 
    plt.title("True Positives ", fontsize=20)
    n_spikes_gt = n_spikes_gt[idx]
    ax1.scatter(ptps[idx], matches_tp/n_spikes_gt, c=clrs[ctr], label=labels[ctr])
    plt.ylim(0, 1.1)
    plt.legend(fontsize=20)
    plt.ylabel("Rates", fontsize=20)
    ax1.tick_params(axis = 'both', which = 'major', labelsize = 20)
    plt.xlabel("PTPs", fontsize=20)

    # 
    ax2=plt.subplot(122)
    plt.title("False Positives ", fontsize=20)
    ax2.scatter(ptps[idx], (n_cmp_spikes-matches_tp)/n_cmp_spikes, c=clrs[ctr], label=labels[ctr])
    plt.legend(fontsize=20)
    plt.ylim(0, 1.1)
    plt.xlim(0,60)
    plt.xlabel("PTPs", fontsize=20)
    ax2.tick_params(axis = 'both', which = 'major', labelsize = 20)

    #plt.title("CMP on time series length: "+str(round((end-start)/20000.,1))+" sec",fontsize=20)

    ctr+=1
plt.suptitle(fname, fontsize=20)
plt.show()
        




/home/cat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/home/cat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
